Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 8.30.0

torch: 2.6.0+cu126



# BatchNorm before and after Activation for Network-in-Network CIFAR-10 Classifier

The CNN architecture is based on  
CNN架构是基于  

- Lin, Min, Qiang Chen, and Shuicheng Yan. "[Network in network](https://arxiv.org/abs/1312.4400)." arXiv preprint arXiv:1312.4400 (2013).  
- 林敏，陈强，颜水成。"[网络中的网络](https://arxiv.org/abs/1312.4400)"。arXiv预印本 arXiv:1312.4400（2013年）。

This paper compares using BatchNorm before the activation function as suggested in  
本文比较了在激活函数之前使用BatchNorm的做法，正如  

- Ioffe, Sergey, and Christian Szegedy. "[Batch normalization: Accelerating deep network training by reducing internal covariate shift.](https://arxiv.org/abs/1502.03167)" arXiv preprint arXiv:1502.03167 (2015)  
- Ioffe, Sergey 和 Christian Szegedy。"[批量归一化：通过减少内部协变量偏移加速深度网络训练。](https://arxiv.org/abs/1502.03167)" arXiv预印本 arXiv:1502.03167（2015年）

and after the activation function as it is nowadays common practice.  
以及现在通常做法中在激活函数之后使用的做法。

## Imports

In [2]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.cuda.set_per_process_memory_fraction(0.3, device=0)

## Model Settings

In [3]:
##########################
### 设置
##########################

# 超参数
RANDOM_SEED = 1  # 随机种子
LEARNING_RATE = 0.0005  # 学习率
BATCH_SIZE = 256  # 批次大小
NUM_EPOCHS = 100  # 训练轮数

# 网络架构
NUM_CLASSES = 10  # 分类类别数

# 其他设置
DEVICE = "cuda:0"  # 使用的设备
GRAYSCALE = False  # 是否使用灰度图像（True 表示使用灰度图像，False 表示使用彩色图像）

In [4]:
##########################
### CIFAR-10 数据集
##########################

# 注意 transforms.ToTensor() 会将输入图像缩放到 0-1 范围内

train_indices = torch.arange(0, 49000)  # 训练集的索引
valid_indices = torch.arange(49000, 50000)  # 验证集的索引

# 加载 CIFAR-10 数据集，train=True 表示训练集，transform=transforms.ToTensor() 将图像转为 Tensor 格式，下载数据集
train_and_valid = datasets.CIFAR10(root='data', 
                                   train=True, 
                                   transform=transforms.ToTensor(),
                                   download=True)

# 从 CIFAR-10 数据集中提取训练集和验证集
train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

# 加载测试集，train=False 表示测试集，transform=transforms.ToTensor() 将图像转为 Tensor 格式
test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())

#####################################################
### 数据加载器
#####################################################

# 创建训练数据加载器，batch_size 为批次大小，num_workers 设置为 8 表示使用 8 个子进程加载数据，shuffle=True 表示打乱数据
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=True)

# 创建验证数据加载器，shuffle=False 表示不打乱验证集数据
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=False)

# 创建测试数据加载器，shuffle=False 表示不打乱测试集数据
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

#####################################################

# 检查数据集的形状
for images, labels in train_loader:  
    print('训练集图像批次的维度:', images.shape)  # 输出训练集图像的维度
    print('训练集标签批次的维度:', labels.shape)  # 输出训练集标签的维度
    break

for images, labels in test_loader:  
    print('测试集图像批次的维度:', images.shape)  # 输出测试集图像的维度
    print('测试集标签批次的维度:', labels.shape)  # 输出测试集标签的维度
    break
    
for images, labels in valid_loader:  
    print('验证集图像批次的维度:', images.shape)  # 输出验证集图像的维度
    print('验证集标签批次的维度:', labels.shape)  # 输出验证集标签的维度
    break

训练集图像批次的维度: torch.Size([256, 3, 32, 32])
训练集标签批次的维度: torch.Size([256])
测试集图像批次的维度: torch.Size([256, 3, 32, 32])
测试集标签批次的维度: torch.Size([256])
验证集图像批次的维度: torch.Size([256, 3, 32, 32])
验证集标签批次的维度: torch.Size([256])


# Without BatchNorm

In [5]:
##########################
### 模型
##########################

class NiN(nn.Module):
    def __init__(self, num_classes):
        super(NiN, self).__init__()
        self.num_classes = num_classes
        # 定义网络的分类器部分
        self.classifier = nn.Sequential(
                # 第一层卷积层：输入通道为 3，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2
                nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第二层卷积层：输入通道为 192，输出通道为 160，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第三层卷积层：输入通道为 160，输出通道为 96，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 最大池化层：卷积后的输出通过最大池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第四层卷积层：输入通道为 96，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2
                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第五层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第六层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第七层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 3，步幅为 1，padding 为 1
                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第八层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第九层卷积层：输入通道为 192，输出通道为 10，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192,  10, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 8，步幅为 1，padding 为 0
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
                )

    def forward(self, x):
        # 前向传播过程
        x = self.classifier(x)  # 通过分类器进行计算
        logits = x.view(x.size(0), self.num_classes)  # 将输出展平，形状为 [batch_size, num_classes]
        probas = torch.softmax(logits, dim=1)  # 使用 softmax 激活函数获得每个类的概率分布
        return logits, probas  # 返回 logits 和概率


In [6]:
torch.manual_seed(RANDOM_SEED)

model = NiN(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [7]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)  # 将特征转移到指定设备
        targets = targets.to(device)  # 将目标标签转移到指定设备

        logits, probas = model(features)  # 前向传播，获取输出的logits和概率
        _, predicted_labels = torch.max(probas, 1)  # 获取最大概率的标签
        num_examples += targets.size(0)  # 更新总样本数
        correct_pred += (predicted_labels == targets).sum()  # 统计预测正确的样本数
    return correct_pred.float()/num_examples * 100  # 返回准确率（百分比）


start_time = time.time()  # 记录开始时间
for epoch in range(NUM_EPOCHS):  # 循环遍历每个epoch
    
    model.train()  # 设置模型为训练模式
    
    for batch_idx, (features, targets) in enumerate(train_loader):  # 循环遍历每个批次
    
        ### 准备小批量数据
        features = features.to(DEVICE)  # 将特征转移到指定设备
        targets = targets.to(DEVICE)  # 将目标标签转移到指定设备
            
        ### 前向传播和反向传播
        logits, probas = model(features)  # 获取模型的输出
        cost = F.cross_entropy(logits, targets)  # 计算交叉熵损失
        optimizer.zero_grad()  # 清空梯度
        
        cost.backward()  # 反向传播计算梯度
        
        ### 更新模型参数
        optimizer.step()  # 使用优化器更新参数
        
        ### 日志记录
        if not batch_idx % 120:  # 每120个批次输出一次日志
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')

    # 计算准确率时不需要构建计算图以进行反向传播
    with torch.set_grad_enabled(False):  # 禁用梯度计算
        train_acc = compute_accuracy(model, train_loader, device=DEVICE)  # 计算训练集准确率
        valid_acc = compute_accuracy(model, valid_loader, device=DEVICE)  # 计算验证集准确率
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')  # 输出准确率
    
    elapsed = (time.time() - start_time)/60  # 计算已经过去的时间（分钟）
    print(f'Time elapsed: {elapsed:.2f} min')  # 输出已用时间
  
elapsed = (time.time() - start_time)/60  # 计算总训练时间（分钟）
print(f'Total Training Time: {elapsed:.2f} min')  # 输出总训练时间


Epoch: 001/100 | Batch 000/192 | Cost: 2.3041
Epoch: 001/100 | Batch 120/192 | Cost: 2.1455
Epoch: 001/100 Train Acc.: 23.27% | Validation Acc.: 23.80%
Time elapsed: 0.69 min
Epoch: 002/100 | Batch 000/192 | Cost: 2.1072
Epoch: 002/100 | Batch 120/192 | Cost: 2.0444
Epoch: 002/100 Train Acc.: 28.35% | Validation Acc.: 30.60%
Time elapsed: 1.39 min
Epoch: 003/100 | Batch 000/192 | Cost: 2.0100
Epoch: 003/100 | Batch 120/192 | Cost: 1.9320
Epoch: 003/100 Train Acc.: 33.56% | Validation Acc.: 34.40%
Time elapsed: 2.08 min
Epoch: 004/100 | Batch 000/192 | Cost: 1.9790
Epoch: 004/100 | Batch 120/192 | Cost: 1.9554
Epoch: 004/100 Train Acc.: 36.74% | Validation Acc.: 39.00%
Time elapsed: 2.76 min
Epoch: 005/100 | Batch 000/192 | Cost: 1.9385
Epoch: 005/100 | Batch 120/192 | Cost: 1.8509
Epoch: 005/100 Train Acc.: 37.33% | Validation Acc.: 39.40%
Time elapsed: 3.45 min
Epoch: 006/100 | Batch 000/192 | Cost: 1.8317
Epoch: 006/100 | Batch 120/192 | Cost: 1.8104
Epoch: 006/100 Train Acc.: 39.69%

In [8]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 50.97%


# BatchNorm before Activation

In [9]:
##########################
### 模型
##########################

class NiN(nn.Module):
    def __init__(self, num_classes):
        super(NiN, self).__init__()
        self.num_classes = num_classes
        # 定义网络的分类器部分
        self.classifier = nn.Sequential(
                # 第一层卷积层：输入通道为 3，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2，bias=False 不使用偏置
                nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2, bias=False),
                nn.BatchNorm2d(192),  # 批归一化层，用于加速训练并稳定训练过程
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第二层卷积层：输入通道为 192，输出通道为 160，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(160),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第三层卷积层：输入通道为 160，输出通道为 96，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(96),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 最大池化层：卷积后的输出通过最大池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第四层卷积层：输入通道为 96，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2，bias=False 不使用偏置
                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2, bias=False),
                nn.BatchNorm2d(192),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第五层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(192),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第六层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(192),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第七层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 3，步幅为 1，padding 为 1，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(192),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第八层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(192),  # 批归一化层
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 第九层卷积层：输入通道为 192，输出通道为 10，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192,  10, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 8，步幅为 1，padding 为 0
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
                )

    def forward(self, x):
        # 前向传播过程
        x = self.classifier(x)  # 通过分类器进行计算
        logits = x.view(x.size(0), self.num_classes)  # 将输出展平，形状为 [batch_size, num_classes]
        probas = torch.softmax(logits, dim=1)  # 使用 softmax 激活函数获得每个类的概率分布
        return logits, probas  # 返回 logits 和概率

In [10]:
torch.manual_seed(RANDOM_SEED)

model = NiN(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [11]:
start_time = time.time()  # 记录开始时间
for epoch in range(NUM_EPOCHS):  # 循环遍历每个epoch
    
    model.train()  # 设置模型为训练模式
    
    for batch_idx, (features, targets) in enumerate(train_loader):  # 循环遍历每个批次
    
        ### 准备小批量数据
        features = features.to(DEVICE)  # 将特征转移到指定设备
        targets = targets.to(DEVICE)  # 将目标标签转移到指定设备
            
        ### 前向传播和反向传播
        logits, probas = model(features)  # 获取模型的输出
        cost = F.cross_entropy(logits, targets)  # 计算交叉熵损失
        optimizer.zero_grad()  # 清空梯度
        
        cost.backward()  # 反向传播计算梯度
        
        ### 更新模型参数
        optimizer.step()  # 使用优化器更新参数
        
        ### 日志记录
        if not batch_idx % 120:  # 每120个批次输出一次日志
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')

    # 计算准确率时不需要构建计算图以进行反向传播
    with torch.set_grad_enabled(False):  # 禁用梯度计算
        train_acc = compute_accuracy(model, train_loader, device=DEVICE)  # 计算训练集准确率
        valid_acc = compute_accuracy(model, valid_loader, device=DEVICE)  # 计算验证集准确率
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')  # 输出准确率
    
    elapsed = (time.time() - start_time)/60  # 计算已经过去的时间（分钟）
    print(f'Time elapsed: {elapsed:.2f} min')  # 输出已用时间
  
elapsed = (time.time() - start_time)/60  # 计算总训练时间（分钟）
print(f'Total Training Time: {elapsed:.2f} min')  # 输出总训练时间

Epoch: 001/100 | Batch 000/192 | Cost: 2.3090
Epoch: 001/100 | Batch 120/192 | Cost: 1.2568
Epoch: 001/100 Train Acc.: 61.16% | Validation Acc.: 61.30%
Time elapsed: 0.82 min
Epoch: 002/100 | Batch 000/192 | Cost: 1.2059
Epoch: 002/100 | Batch 120/192 | Cost: 0.9759
Epoch: 002/100 Train Acc.: 69.01% | Validation Acc.: 67.20%
Time elapsed: 1.65 min
Epoch: 003/100 | Batch 000/192 | Cost: 0.9362
Epoch: 003/100 | Batch 120/192 | Cost: 0.9533
Epoch: 003/100 Train Acc.: 73.57% | Validation Acc.: 72.00%
Time elapsed: 2.47 min
Epoch: 004/100 | Batch 000/192 | Cost: 0.9013
Epoch: 004/100 | Batch 120/192 | Cost: 0.7463
Epoch: 004/100 Train Acc.: 75.14% | Validation Acc.: 73.60%
Time elapsed: 3.30 min
Epoch: 005/100 | Batch 000/192 | Cost: 0.7775
Epoch: 005/100 | Batch 120/192 | Cost: 0.7081
Epoch: 005/100 Train Acc.: 78.69% | Validation Acc.: 75.70%
Time elapsed: 4.13 min
Epoch: 006/100 | Batch 000/192 | Cost: 0.6839
Epoch: 006/100 | Batch 120/192 | Cost: 0.6343
Epoch: 006/100 Train Acc.: 79.81%

In [12]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 85.12%


# BatchNorm after Activation

In [13]:
##########################
### 模型
##########################

class NiN(nn.Module):
    def __init__(self, num_classes):
        super(NiN, self).__init__()
        self.num_classes = num_classes
        # 定义网络的分类器部分
        self.classifier = nn.Sequential(
                # 第一层卷积层：输入通道为 3，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2，bias=False 不使用偏置
                nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(192),  # 批归一化层，用于加速训练并稳定训练过程
                
                # 第二层卷积层：输入通道为 192，输出通道为 160，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(160),  # 批归一化层
                
                # 第三层卷积层：输入通道为 160，输出通道为 96，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(96),  # 批归一化层
                
                # 最大池化层：卷积后的输出通过最大池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第四层卷积层：输入通道为 96，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2，bias=False 不使用偏置
                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(192),  # 批归一化层
                
                # 第五层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(192),  # 批归一化层
                
                # 第六层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(192),  # 批归一化层
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第七层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 3，步幅为 1，padding 为 1，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(192),  # 批归一化层
                
                # 第八层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                nn.BatchNorm2d(192),  # 批归一化层
                
                # 第九层卷积层：输入通道为 192，输出通道为 10，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192,  10, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 8，步幅为 1，padding 为 0
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
                )

    def forward(self, x):
        # 前向传播过程
        x = self.classifier(x)  # 通过分类器进行计算
        logits = x.view(x.size(0), self.num_classes)  # 将输出展平，形状为 [batch_size, num_classes]
        probas = torch.softmax(logits, dim=1)  # 使用 softmax 激活函数获得每个类的概率分布
        return logits, probas  # 返回 logits 和概率

In [14]:
torch.manual_seed(RANDOM_SEED)

model = NiN(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [15]:
start_time = time.time()  # 记录开始时间
for epoch in range(NUM_EPOCHS):  # 循环遍历每个epoch
    
    model.train()  # 设置模型为训练模式
    
    for batch_idx, (features, targets) in enumerate(train_loader):  # 循环遍历每个批次
    
        ### 准备小批量数据
        features = features.to(DEVICE)  # 将特征转移到指定设备
        targets = targets.to(DEVICE)  # 将目标标签转移到指定设备
            
        ### 前向传播和反向传播
        logits, probas = model(features)  # 获取模型的输出
        cost = F.cross_entropy(logits, targets)  # 计算交叉熵损失
        optimizer.zero_grad()  # 清空梯度
        
        cost.backward()  # 反向传播计算梯度
        
        ### 更新模型参数
        optimizer.step()  # 使用优化器更新参数
        
        ### 日志记录
        if not batch_idx % 120:  # 每120个批次输出一次日志
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')

    # 计算准确率时不需要构建计算图以进行反向传播
    with torch.set_grad_enabled(False):  # 禁用梯度计算
        train_acc = compute_accuracy(model, train_loader, device=DEVICE)  # 计算训练集准确率
        valid_acc = compute_accuracy(model, valid_loader, device=DEVICE)  # 计算验证集准确率
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')  # 输出准确率
    
    elapsed = (time.time() - start_time)/60  # 计算已经过去的时间（分钟）
    print(f'Time elapsed: {elapsed:.2f} min')  # 输出已用时间
  
elapsed = (time.time() - start_time)/60  # 计算总训练时间（分钟）
print(f'Total Training Time: {elapsed:.2f} min')  # 输出总训练时间

Epoch: 001/100 | Batch 000/192 | Cost: 2.3065
Epoch: 001/100 | Batch 120/192 | Cost: 1.2000
Epoch: 001/100 Train Acc.: 63.83% | Validation Acc.: 62.40%
Time elapsed: 0.83 min
Epoch: 002/100 | Batch 000/192 | Cost: 1.1581
Epoch: 002/100 | Batch 120/192 | Cost: 0.9258
Epoch: 002/100 Train Acc.: 71.11% | Validation Acc.: 69.10%
Time elapsed: 1.65 min
Epoch: 003/100 | Batch 000/192 | Cost: 0.9431
Epoch: 003/100 | Batch 120/192 | Cost: 0.9069
Epoch: 003/100 Train Acc.: 75.49% | Validation Acc.: 73.20%
Time elapsed: 2.48 min
Epoch: 004/100 | Batch 000/192 | Cost: 0.8423
Epoch: 004/100 | Batch 120/192 | Cost: 0.6647
Epoch: 004/100 Train Acc.: 78.95% | Validation Acc.: 75.90%
Time elapsed: 3.30 min
Epoch: 005/100 | Batch 000/192 | Cost: 0.6592
Epoch: 005/100 | Batch 120/192 | Cost: 0.5978
Epoch: 005/100 Train Acc.: 81.32% | Validation Acc.: 79.40%
Time elapsed: 4.13 min
Epoch: 006/100 | Batch 000/192 | Cost: 0.5774
Epoch: 006/100 | Batch 120/192 | Cost: 0.6013
Epoch: 006/100 Train Acc.: 83.19%

In [16]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 84.85%


In [17]:
%watermark -iv

PIL        : 11.0.0
pandas     : 2.2.3
torchvision: 0.21.0+cu126
matplotlib : 3.10.1
numpy      : 2.1.2
torch      : 2.6.0+cu126

